## Current Python Version

Ensure the python version running in the jupyter kernel is the same as expected

In [1]:
# from platform import python_version

# print(python_version())


List installed packages, should match the ones in the pipfile (and their dependencies)

In [2]:
# pip list

## Internal Module Imports
Code required to enable project's internally defined modulos into the jupyter notebook environment

In [3]:
import tensorflow as tf

In [4]:
# tf.test.is_built_with_cuda()
with tf.Session() as sess:
    devices = sess.list_devices()
    print(devices)

# tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 3603270731921763664), _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 3148847513, 5145087521365898564)]


In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5650095819020802393
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3148847513
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11760594832903787893
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [6]:
# tf.test.gpu_device_name()


In [7]:
# import tensorflow as tf
# print(tf.reduce_sum(tf.random_normal([1000, 1000])))

In [8]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [9]:
# from electricity import ElectricalSystem, Load, Generator
from dto import (
    COST_PRESETS, 
    ElectricalSystemSpecs, 
    EpsilonSpecs,
    GeneratorSpecs, 
    LoadSpecs, 
    NodeStatePower, 
    SystemHistory, 
    )
    
from learning import ModelTrainer, ModelTester, LearningParams

In [10]:
import tensorflow as tf
tf.test.is_gpu_available()

True

Determine the experiment params

In [11]:
from dto import EpsilonSpecs, ElectricalConstants
import math

LearningParams(
    gamma=0.9,   # Gamma (Discount)
    tau=0.001, # Tau
    epsilonSpecs = EpsilonSpecs( # Epsilon explore/exploit control
        thresholdProgress = 0.6, # % of steps where decay change should happen
        thresholdValue = 0.5, # Value at which decay change would happen
        finalValue = 0.0001, # Value at the end of the experiment
    ),
    numEpisodes=5000, # Number of learning episodes to run
    maxSteps=150,   # Number of steps per learning episode
    traceLength=8,     # Number of steps each sampled episode should contain
    batchSize=4,     # Number of episodes sampled from experience buffer
    electricalSystemSpecs = ElectricalSystemSpecs(
        loads=[LoadSpecs(id_="L1", basePower=3.0, noiseLevel=0.1)],
        generators=[
            GeneratorSpecs(id_="G1", basePower=1.0, costProfile=COST_PRESETS.COAL, minPower=0.5, maxPower=2.0, noiseLevel=0),
            GeneratorSpecs(id_="G2", basePower=1.0, costProfile=COST_PRESETS.OIL, minPower=0.5, maxPower=2.0, noiseLevel=0),
#             GeneratorSpecs(id_="G3", basePower=1.0, costProfile=COST_PRESETS.OIL, minPower=0.5, maxPower=2.0, noiseLevel=0),
#             GeneratorSpecs(id_="G4", basePower=1.0, costProfile=COST_PRESETS.OIL, minPower=0.5, maxPower=2.0, noiseLevel=0),
        ],
        shouldTrackOptimalCost=True,
    ),
    modelName='CUDA-Test-Frequency-3-generators-3'
)

# Droop = +infinity essentially remove the droop component from the power update equation. System reacts as if droop is disabled
# ElectricalConstants().droop = math.inf

LearningParams(electricalSystemSpecs=ElectricalSystemSpecs(loads=[LoadSpecs(id_='L1', basePower=3.0, noiseLevel=0.1)], generators=[GeneratorSpecs(id_='G1', basePower=1.0, costProfile=CostProfile(alpha=510.0, beta=7.2, gamma=0.00142), minPower=0.5, maxPower=2.0, noiseLevel=0), GeneratorSpecs(id_='G2', basePower=1.0, costProfile=CostProfile(alpha=310.0, beta=7.85, gamma=0.00194), minPower=0.5, maxPower=2.0, noiseLevel=0), GeneratorSpecs(id_='G3', basePower=1.0, costProfile=CostProfile(alpha=310.0, beta=7.85, gamma=0.00194), minPower=0.5, maxPower=2.0, noiseLevel=0)], shouldTrackOptimalCost=True), gamma=0.9, tau=0.001, epsilonSpecs=EpsilonSpecs(thresholdProgress=0.6, thresholdValue=0.5, finalValue=0.0001), numEpisodes=5000, maxSteps=150, traceLength=8, batchSize=4, nnShape=NeuralNetworkShape(layer_00_ltsm=100, layer_01_mlp_01=1000, layer_02_mlp_02=100, layer_03_mlp_03=50, layer_04_mlp_04=1), modelName='CUDA-Test-Frequency-3-generators-3')

Train Agents

In [12]:
def rewardFn(deltaFreq):
    baseComponent = 2**(-(deltaFreq**2)/100)
    peakComponent = 2**(-(deltaFreq**2)/2)
    earnedReward = baseComponent + 9*peakComponent

    return earnedReward, {'base': baseComponent, 'peak': 9*peakComponent, 'total':earnedReward}

In [13]:
from learning.frequency import ModelAdapterFrequency as ModelAdapter


In [14]:
%%time
modelAdapter = ModelAdapter(rewardFn=rewardFn)
allAgents = ModelTrainer(modelAdapter).trainAgents()

W0915 21:00:33.273503 15384 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0915 21:00:33.273503 15384 deprecation.py:323] From C:\Users\flavi\Documents\Study\City-UniLondon\Thesis\2019-MSc_Thesis\app\learning\actor.py:29: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0915 21:00:33.291455 15384 deprecation.py:323] From C:\Users\flavi\Documents\Study\City-UniLondon\Thesis\2019-MSc_Thesis\app\learning\actor.py:38: dynamic_rnn (from tensorflow.python.ops.rn

agent G1: actor
agent G1: actor-target
  critic cost_freq_G1_critic: 1
  critic cost_freq_G1_critic: 2
  critic cost_freq_G1_critic: 3
  critic cost_freq_G1_critic: 4
  critic cost_freq_G1_critic: 5
  critic cost_freq_G1_critic: 6
  critic cost_freq_G1_critic: 7
  critic cost_freq_G1_critic: 8
agent G1: critic
  critic cost_freq_G1_critic_target: 1
  critic cost_freq_G1_critic_target: 2
  critic cost_freq_G1_critic_target: 3
  critic cost_freq_G1_critic_target: 4
  critic cost_freq_G1_critic_target: 5
  critic cost_freq_G1_critic_target: 6
  critic cost_freq_G1_critic_target: 7
  critic cost_freq_G1_critic_target: 8
agent G1: critic-target
agent G2: actor
agent G2: actor-target
  critic cost_freq_G2_critic: 1
  critic cost_freq_G2_critic: 2
  critic cost_freq_G2_critic: 3
  critic cost_freq_G2_critic: 4
  critic cost_freq_G2_critic: 5
  critic cost_freq_G2_critic: 6
  critic cost_freq_G2_critic: 7
  critic cost_freq_G2_critic: 8
agent G2: critic
  critic cost_freq_G2_critic_target: 1
 

KeyboardInterrupt: 

In [15]:
STEPS_TO_TEST = 300

# testSpecs = ElectricalSystemSpecs(
#         loads=[LoadSpecs(id_="L1", basePower=3.0, noiseLevel=0.1)],
#         generators=[
#             GeneratorSpecs(id_="G1", basePower=1.5, costProfile=COST_PRESETS.COAL, minPower=0.5, maxPower=2.0, noiseLevel=0.0),
#             GeneratorSpecs(id_="G2", basePower=1.5, costProfile=COST_PRESETS.OIL, minPower=0.5, maxPower=2.0, noiseLevel=0.0),
#         ],
# )
# electricalSystem, rewards = ModelTester.testAgents(testSpecs, LearningParams().modelName, STEPS_TO_TEST)


electricalSystem, rewards = ModelTester(modelAdapter).testAgents(LearningParams().electricalSystemSpecs, LearningParams().modelName, STEPS_TO_TEST)

agent G1: actor
agent G1: actor-target
  critic cost_freq_G1_critic: 1
  critic cost_freq_G1_critic: 2
  critic cost_freq_G1_critic: 3
  critic cost_freq_G1_critic: 4


KeyboardInterrupt: 

Alternatively, one can plot the data to better inspect the progression

In [ ]:
import matplotlib.pyplot as plt
from plots import plotAll, plotExperimentRewardProgression

In [ ]:
plotExperimentRewardProgression(LearningState().model.allRewards)

Plot the reward progression of the model testing

In [ ]:
steps = range(STEPS_TO_TEST)
plt.scatter(steps, rewards)
sum(rewards)

Plot all graphs related to the electrical system state during the model testing

In [ ]:
plotAll(electricalSystem.systemHistory)

In [ ]:
# # Show the last episode ran during training
from learning.learning_state import LearningState
plotAll(LearningState().episode.electricalSystem.systemHistory)

In [ ]:
from learning.learning_state import LearningState
import numpy as np

rewList = LearningState().model.cummRewardList

In [ ]:
LearningState().model.allRewards

In [ ]:
allRewards = LearningState().model.allRewards


for r in allRewards:
    plt.figure()
    plt.scatter(range(len(r)), r)
# sum(rewards)

In [ ]:
plt.scatter(range(len(rewList)), rewList)

In [ ]:
from plots.plot_constants import COLOR_PALETTE, FONT_SIZES

history = electricalSystem.systemHistory
# Get series to be plotted
stepsSeries = history.steps
loads = history.loads
totalLoadSeries = history.totalLoad
generators = history.generators
totalPowerSeries = history.totalPower

plt.figure(0)

# Declare colors to be used
colorTotalLoad = COLOR_PALETTE[0]
colorTotalPower = COLOR_PALETTE[1]
colorsIndividualNodes = COLOR_PALETTE[3:]

# Plot total power/load data
plt.plot(stepsSeries, totalLoadSeries, color=colorTotalLoad)
plt.plot(stepsSeries, totalPowerSeries, color=colorTotalPower)
legendFields = ['Total Load', 'Total Power']

# Multiple scenarios involve a single load, so we make it optional to plot that single load
if(False):
    for idx, loadId in enumerate(loads):
        # Since num loads is variable, colors may wrap around the palette
        loadColor = colorsIndividualNodes[idx % len(colorsIndividualNodes)]
        loadLegend = loadId
        loadSeries = loads[loadId]
        plt.plot(stepsSeries, loadSeries, color=loadColor, linestyle='--')
        legendFields.append(loadLegend)

for idx, generatorId in enumerate(generators):
    # Since num generators is variable, colors may wrap around the palette
    generatorColor = colorsIndividualNodes[idx % len(colorsIndividualNodes)]
    generatorLegend = generatorId
    generatorSeries = generators[generatorId]
    plt.plot(stepsSeries, generatorSeries, color=generatorColor)
    legendFields.append(generatorLegend)
    

summedPow = [x + y for x, y in zip(generators.get('G1'), generators.get('G2'))]
plt.plot(stepsSeries, summedPow, color=COLOR_PALETTE[2])
legendFields.append("Summed Pow")


plt.legend(legendFields)
plt.xlabel('Steps', fontsize=FONT_SIZES['AXIS_LABEL'])
plt.ylabel('Power (pu)', fontsize=FONT_SIZES['AXIS_LABEL'])

plt.title('System Power (pu) x Time (Steps)', fontsize=FONT_SIZES['TITLE'])

plt.show()